In [ ]:
import os
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install boto3
import io
import os
import tarfile
import tempfile

import boto3
import matplotlib.pyplot as plt
import requests
from botocore import UNSIGNED
from botocore.config import Config
from IPython.display import Audio
from torchaudio.utils import download_asset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import librosa
import librosa.display
import IPython.display

### 1. 데이터셋 구축

In [ ]:
main_df = pd.read_csv('/content/drive/MyDrive/23-2_speaker_verification/dataset_new/main_df.csv')
main_df

In [ ]:
main_df['speaker_code'].value_counts()

'''
1580 : F
61 : M로 테스트

파일 이름, speaker_code

'''

4446    108
1580    105
61      104
5142    102
3752    101
       ... 
1221     41
4077     39
2902     38
422      36
8224     32
Name: speaker_code, Length: 80, dtype: int64

In [ ]:
F_1580 = main_df[main_df['speaker_code'] == 1580][:104][['파일 이름']]
F_1580['label'] = 0

In [ ]:
M_61 = main_df[main_df['speaker_code'] == 61][:104][['파일 이름']]
M_61['label'] = 1

In [ ]:
si_df = pd.concat([F_1580, M_61], axis = 0)
si_df.reset_index()

,index,파일 이름,label
0,2712,1580-141084-0031.flac,0
1,2725,1580-141083-0036.flac,0
2,2776,1580-141084-0027.flac,0
3,2779,1580-141083-0020.flac,0
4,2954,1580-141084-0050.flac,0
...,...,...,...
203,5203,61-70968-0053.flac,1
204,5214,61-70970-0002.flac,1
205,5226,61-70968-0004.flac,1
206,5265,61-70968-0028.flac,1


In [ ]:
si_df.to_csv('/content/drive/MyDrive/23-2_kubig_confer/dataset/si_test_df.csv',index=False)

In [ ]:
# 침풍
main_df = pd.read_csv('/content/drive/MyDrive/23-2_kubig_confer/dataset/chim_pung.csv')
main_df

,index,file,label
0,81,chimchak-1.wav,0
1,132,chimchak-10.wav,0
2,177,chimchak-100.wav,0
3,135,chimchak-11.wav,0
4,126,chimchak-12.wav,0
...,...,...,...
195,199,kim_pung-95.wav,1
196,184,kim_pung-96.wav,1
197,170,kim_pung-97.wav,1
198,0,kim_pung-98.wav,1


### 2. 데이터로더 생성

In [ ]:
MFCC_X = 20
MFCC_Y = 800 # 11초 이하로 해도 512하면 짤림

In [ ]:

def _normalize(mfcc):
  return (mfcc - np.min(mfcc)) / (np.max(mfcc) - np.min(mfcc))


def _fit_size(arr, size):
  zeros = np.zeros( (len(arr), size - len(arr[0])))
  arr = np.append(arr, zeros, axis = 1)
  arr.astype('float32')
  return arr


def _mfcc(path):
  y, sr = librosa.load(path)
  mfcc = librosa.feature.mfcc(y = y, sr = sr)
  mfcc = _fit_size(mfcc,MFCC_Y)
  return _normalize(mfcc)


def input_tran(tensor):
  tensor = tensor.view(1,10,20,800)
  tensor = tensor.permute(1,0,2,3) # NCHW
  tensor = tensor.float() # float 변환
  return tensor


In [ ]:
test_df

,index,file,label
0,81,chimchak-1.wav,0
1,132,chimchak-10.wav,0
2,177,chimchak-100.wav,0
3,135,chimchak-11.wav,0
4,126,chimchak-12.wav,0
5,125,chimchak-13.wav,0
6,110,chimchak-14.wav,0
7,109,chimchak-15.wav,0
8,118,chimchak-16.wav,0
9,119,chimchak-17.wav,0


In [ ]:
train_df = main_df[30:180]
test_df = pd.concat([main_df[0:29], main_df[180:199]]) # 7:3 split

In [ ]:
# 셔플 후 train, test 분리

train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
# train_df = shuffled_df[:154]
# test_df = shuffled_df[154:].reset_index(drop=True)

In [ ]:

class CustomDataset(Dataset):
  def __init__(self, train = True, train_df = train_df, test_df = test_df):
    self.train = train
    num_train = len(train_df)
    num_test = len(test_df)

    if self.train == True:
      self.train_data = []
      self.train_label = []

      print("===========train Data===========")
      for i in range(num_train):
        path1 = ( '/content/drive/MyDrive/23-2_kubig_confer/dataset/dataset_all2/' + train_df['file'][i])
        mfcc = _mfcc(path1)
        self.train_data.append(mfcc)
        self.train_label.append(train_df['label'][i])

        if i % 10 == 0:
          print('Train DataLoader ' , i , '번째 완료')

      self.train_label = np.array(self.train_label)
      self.train_data = np.concatenate(self.train_data)
      self.train_data = self.train_data.reshape(num_train, MFCC_X, MFCC_Y)
    else:
      self.test_data = []
      self.test_label = []

      print('===========test Data=============')
      for i in range(num_test):
        path = ('/content/drive/MyDrive/23-2_kubig_confer/dataset/dataset_all2/' + test_df['file'][i])
        mfcc = _mfcc(path)
        self.test_data.append(mfcc)
        self.test_label.append(test_df['label'][i])

        if i % 10 == 0:
          print('Test DataLoader ' , i , '번째 완료')

  def __getitem__(self,index):
    if self.train:
      return self.train_data[index], self.train_label[index]
    else:
      return self.test_data[index], self.test_label[index]

  def __len__(self):
    if self.train:
      return len(self.train_data)
    else:
      return len(self.test_data)



In [ ]:

train_dataset = CustomDataset(train = True)
test_dataset = CustomDataset(train = False)

train_loader = DataLoader(train_dataset, batch_size = 10, shuffle = True, num_workers = 0, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size = 10, shuffle = False, num_workers = 0, drop_last = True)



===========train Data===========
Train DataLoader  0 번째 완료
Train DataLoader  10 번째 완료
Train DataLoader  20 번째 완료
Train DataLoader  30 번째 완료
Train DataLoader  40 번째 완료
Train DataLoader  50 번째 완료
Train DataLoader  60 번째 완료
Train DataLoader  70 번째 완료
Train DataLoader  80 번째 완료
Train DataLoader  90 번째 완료
Train DataLoader  100 번째 완료
Train DataLoader  110 번째 완료
Train DataLoader  120 번째 완료
Train DataLoader  130 번째 완료
Train DataLoader  140 번째 완료
===========test Data=============
Test DataLoader  0 번째 완료
Test DataLoader  10 번째 완료
Test DataLoader  20 번째 완료
Test DataLoader  30 번째 완료
Test DataLoader  40 번째 완료


In [ ]:
# dataloader test code

cnt = 0
for [train_data, train_label] in train_loader:
  print(train_data.shape)
  print(train_label.shape)
  print(train_label)
  cnt += 1
  # if cnt == 10:
  #   break

cnt = 0
for [test_data, test_label] in test_loader:
  print(test_data.shape)
  print(test_label.shape)
  print(test_label)
  cnt += 1
  # if cnt == 10:
  #   break


torch.Size([10, 20, 800])
torch.Size([10])
tensor([1, 1, 0, 1, 0, 0, 0, 1, 0, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([0, 1, 1, 0, 0, 0, 1, 1, 0, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([1, 0, 1, 0, 1, 0, 1, 0, 0, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([0, 1, 0, 1, 1, 1, 1, 0, 0, 0])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([1, 1, 0, 1, 1, 0, 0, 1, 0, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([1, 0, 1, 1, 1, 0, 0, 1, 0, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([0, 0, 1, 0, 0, 0, 1, 0, 0, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([1, 0, 0, 0, 1, 1, 1, 0, 0, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([1, 0, 0, 1, 1, 0, 1, 1, 1, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([1, 0, 1, 1, 1, 0, 0, 1, 1, 1])
torch.Size([10, 20, 800])
torch.Size([10])
tensor([0, 1, 0, 1, 0, 1, 0, 0, 0, 1])
torch.Size([10, 

### 3. 모델링

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 2 outputs
        self.fc = torch.nn.Linear(5 * 200 * 64, 2, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out


In [ ]:

model = CNN()

In [ ]:

learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 15


In [ ]:
for epoch in range(30): # total 45
    avg_cost = 0

    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = input_tran(X)
        Y = Y

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.0029494022
[Epoch:    2] cost = 0.00284902984
[Epoch:    3] cost = 0.00286950823
[Epoch:    4] cost = 0.00270962552
[Epoch:    5] cost = 0.00270984857
[Epoch:    6] cost = 0.0025558495
[Epoch:    7] cost = 0.00234775292
[Epoch:    8] cost = 0.0024258499
[Epoch:    9] cost = 0.00230565877
[Epoch:   10] cost = 0.00214230479
[Epoch:   11] cost = 0.00215390185
[Epoch:   12] cost = 0.00224497751
[Epoch:   13] cost = 0.00221816031
[Epoch:   14] cost = 0.00194573705
[Epoch:   15] cost = 0.00187002821
[Epoch:   16] cost = 0.00183116226
[Epoch:   17] cost = 0.00179323938
[Epoch:   18] cost = 0.00179402204
[Epoch:   19] cost = 0.00167199504
[Epoch:   20] cost = 0.00175899663
[Epoch:   21] cost = 0.00162489596
[Epoch:   22] cost = 0.00173581811
[Epoch:   23] cost = 0.00151905615
[Epoch:   24] cost = 0.00147293264
[Epoch:   25] cost = 0.00141977321
[Epoch:   26] cost = 0.0013925099
[Epoch:   27] cost = 0.00135952444
[Epoch:   28] cost = 0.00132601871
[Epoch:   29] cost = 0.0

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
acc = 0
with torch.no_grad():

  for X_test, Y_test in test_loader:
    X_test = input_tran(X_test)
    Y_test = Y_test
    prediction = model(X_test)
    accuracy = torch.argmax(prediction, 1) == Y_test
    acc += accuracy.float().mean()

  print('Accuracy:', acc / len(test_loader))

  # 30 epoch에 0.90
  # 45 epoch에 0.925
  # 75 epoch에 0.950
  # 105 epoch에 0.950

Accuracy: tensor(0.9500)
